## RunnableLambda 객체 사용방법

In [2]:
from langchain_core.runnables import RunnableLambda

# 함수 정의
def func(x):
    return  x*2

# 함수를 전달인자로 넣기
runnable_1 = RunnableLambda(func)

# RunnableLambda를 통한 함수 실행
print(runnable_1.invoke(10))

20


In [2]:
# RunnableLambda 객체 사용
runnable_1 = RunnableLambda(lambda x: x*2)
print(runnable_1.invoke(10))

20


In [3]:
# runnable batch
runnable_3 = RunnableLambda(func)

print(runnable_3.batch([10, 20, 30]))

[20, 40, 60]


- 함수 값을 return 으로 반환

In [4]:
def gen(x):
    for y in x :
        return y * 2  
    

# runnable 
runnable_4 = RunnableLambda(gen)
print(type(runnable_4.stream(range(10))))
print(runnable_4.stream(range(10)))

for chunk in runnable_4.stream(range(10)):
    print(type(chunk))
    print(chunk)


<class 'generator'>
<generator object RunnableLambda.transform at 0x000001F69E6C9040>
<class 'int'>
0


- yield로 유지

In [ ]:
def gen(x):
    for y in x :
        yield y * 2   # 값을 반환하지만 함수는 계속 유지 (return은 반환하고 종료)

# runnable 
runnable_4 = RunnableLambda(gen)
for chunk in runnable_4.stream(range(10)):
    print(chunk)


0
2
4
6
8
10
12
14
16
18


## 순차적으로 실행하게 하기

In [15]:
from langchain_core.runnables import RunnableLambda, RunnableSequence

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 2)

chain = r1 | r2

chain.invoke(10)



32

# 

In [16]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 2)

# chain = r1 | r2  # 아래와 같은 표현
chain = RunnableSequence(r1, r2)

chain.invoke(10)

32

In [9]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 2)

# 병렬처리 , r1, r2 키는 임으로 설정해 주면 됨.
# chain = RunnableParallel(r1=r1, r2=r2)
chain = RunnableParallel(first=r1, second=r2)

chain.invoke(10)

{'first': 30, 'second': 12}

In [22]:
# 그래프 그리기 알고리즘 
%pip install -qU grandalf

Note: you may need to restart the kernel to use updated packages.


In [7]:
[{"foo":2}]*3

[{'foo': 2}, {'foo': 2}, {'foo': 2}]

In [30]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

runnable_1 = RunnableLambda(lambda x: {"foo": x})
runnable_2 = RunnableLambda(lambda x: [x] * 3)
runnable_3 = RunnableLambda(lambda x: str(x))

chain = runnable_1 | RunnableParallel(r2=runnable_2, r3=runnable_3)

print(chain.invoke(2))
chain.get_graph().print_ascii()

{'r2': [{'foo': 2}, {'foo': 2}, {'foo': 2}], 'r3': "{'foo': 2}"}
        +-------------+        
        | LambdaInput |        
        +-------------+        
               *               
               *               
               *               
          +--------+           
          | Lambda |           
          +--------+           
               *               
               *               
               *               
   +----------------------+    
   | Parallel<r2,r3>Input |    
   +----------------------+    
          *         *          
        **           **        
       *               *       
+--------+          +--------+ 
| Lambda |          | Lambda | 
+--------+          +--------+ 
          *         *          
           **     **           
             *   *             
  +-----------------------+    
  | Parallel<r2,r3>Output |    
  +-----------------------+    


## LCEL 문법 적용

- 방법1

In [24]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [25]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
    다음 컨텍스트에 대해서만 답하세요.
    컨텍스트 :
    {context}
    질문:
    {query}
""")

chain = prompt | llm
input = {
            "context": "Vector Search에 의한 컨텐스트 내용", 
            "query": "안녕"
        }
print(chain.invoke(input))

content='안녕하세요! 벡터 검색과 관련하여 궁금한 점이 있으면 언제든지 물어보세요. 어떻게 도와드릴까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 42, 'total_tokens': 74, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BkKAzEVLARudAe0AAkQxNd2XeaPEj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--f826554e-bc9d-4ead-afc8-e097bc6ee8ec-0' usage_metadata={'input_tokens': 42, 'output_tokens': 32, 'total_tokens': 74, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


- 방법2 RunnableLambda 사용

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

prompt = ChatPromptTemplate.from_template("""
    다음 컨텍스트에 대해서만 답하세요.
    컨텍스트 :
    {context}
    질문:
    {query}
""")

# 기본값을 제공하는 함수
def add_default_values(input_dict):
    return {
        "context": input_dict.get("context", "Vector Search에 의한 컨텐스트 내용"),
        "query": input_dict.get("query", "안녕")
    }


# 체인 구성
chain = RunnableLambda(add_default_values) | prompt | llm


In [27]:
# 빈 딕셔너리로도 실행 가능
result = chain.invoke({})
print(result)

content='안녕하세요! 어떻게 도와드릴까요? Vector Search에 대해 궁금한 점이 있으신가요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 42, 'total_tokens': 66, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a288987b44', 'id': 'chatcmpl-BkKB37b5dwnmKkPEY4dZ0lJHGd6WI', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--3d2b5caf-1574-4c0b-b1a4-6da65520da40-0' usage_metadata={'input_tokens': 42, 'output_tokens': 24, 'total_tokens': 66, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [28]:
# 또는 일부 값만 override
result2 = chain.invoke({"query": "RunnableLambda에 대해 1문장으로 설명해줘"})
print(result2)

content='RunnableLambda는 자바에서 람다식을 사용하여 Runnable 인터페이스의 인스턴스를 간결하게 생성할 수 있게 해주는 표현 방식입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 52, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BkKB5ZCC2AfptjAsygqmolMgkSbPm', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--fe492fd7-5839-4a52-a6af-c051a852c1f0-0' usage_metadata={'input_tokens': 52, 'output_tokens': 34, 'total_tokens': 86, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


- Runnable 객체를 만들면 pipeline에 태울 수 있다는 것을 아는게 중요함.

# 실행을 그래프로 시각화 하기

In [29]:
chain.get_graph().print_ascii()

+--------------------------+ 
| add_default_values_input | 
+--------------------------+ 
              *              
              *              
              *              
   +--------------------+    
   | add_default_values |    
   +--------------------+    
              *              
              *              
              *              
   +--------------------+    
   | ChatPromptTemplate |    
   +--------------------+    
              *              
              *              
              *              
       +------------+        
       | ChatOpenAI |        
       +------------+        
              *              
              *              
              *              
    +------------------+     
    | ChatOpenAIOutput |     
    +------------------+     
